# 隐式马尔可夫模型基础

## 摘要

我们如何将机器学习应用在基于时间序列的观测数据上？举个例子，比如我们可能想要从某人演讲的音频记录中得到他使用的词语序列；或者根据词性标注对一系列单词做注解等等。在本文中，我们将对马尔可夫模型（在时间序列上的状态的形式化推理）以及隐式马尔可夫模型（从观测数据的序列中找回原来的状态序列）的概念从数学方面做一个全面的介绍。本文的最后一节包含了本文中某些观点的来源并提供深入学习的材料。

## 1. 马尔可夫模型

对于给定的状态集$S=\left\{s_1,s_2,\cdots,s_{\lvert S\rvert}\right\}$，我们可以沿时间序列观测到一些列数据$\vec z\in S^T$。举个例子，一个从天气系统中得到的状态集$S=\left\{sun,cloud,rain\right\}$，集合的大小为$\lvert S\rvert=3$，我们也在天气系统中得到一组观测数据$\left\{z_1=s_{sun},z_1=s_{cloud},z_1=s_{cloud},z_1=s_{rain},z_1=s_{cloud},\right\}$，观测数据集大小为$T=5$。

天气例子的的观测状态向我们展示了一个在时间上的随机过程的输出。如果不做进一步假设，$t$时刻的状态$s_j$可能是一个关于任意多个变量的函数，其中包括了从时刻$1$到$t-1$的所有状态、以及一些我们甚至都没有建模的变量。不过，我们可以做出两个能够使时间序列更已处理的**马尔可夫假设（Markov Assumptions）**。

**限制时域假设（Limited Horizon Assumption）**指的是到达$t$时刻状态的概率只与$t-1$时刻的状态有关。这条假设的直观理解就是$t$时刻的状态“足以概括”之前所有状态，并可以合理的预测未来的状态，形式化的表示为：
$$
P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1\right)=P\left(z_t\mid z_{t-1}\right)
$$

**平稳过程假设（Stationary Process Assumption）**指的是在当前状态下，下个状态的条件分布不随时间变化而改变，形式化的表示为：
$$
P\left(z_t\mid z_{t-1}\right)=P\left(z_2\mid z_1\right);\ t\in2\cdots T
$$

按照习惯，我们也会假设初始状态和初始观测数据满足$z_0\equiv s_0$，$s_0$表示在$0$时刻的状态的初始概率分布。这个习惯可以使我们将“观测第一个真实状态$z_1$的先验概率”的观点通过$P\left(z_1\mid z_0\right)$表达出来。因为我们定义对于任意状态序列都有$z_0=s_0$，所以$P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1\right)=P\left(z_t\mid z_{t-1},z_{t-2},\cdots,z_1,z_0\right)$。（其他HMM的表示方法有时通过向量$\pi\in\mathbb R^{\lvert S\rvert}$表达其先验的观点。）

我们定义一个状态转换矩阵$\displaystyle A\in\mathbb R^{(\lvert S\rvert+1)\times(\lvert S\rvert+1)}$来参数化这些转换，$A_{ij}$就表示在任意时刻$t$下从状态$i$转换到状态$j$的概率。（可以参考[线性代数中马尔科夫矩阵的例子](http://nbviewer.jupyter.org/github/zlotus/notes-linear-algebra/blob/master/chapter24.ipynb)）在天气系统的例子中，就可以定义如下的状态转换矩阵：
$$
A=\begin{matrix}
&s_0&s_{sun}&s_{cloud}&s_{rain}\\
s_0&0&.33&.33&.33\\
s_{cloud}&0&.2&.6&.2\\
s_{rain}&0&.1&.2&.7
\end{matrix}
$$

容易观察到这些数字（虽然是编造的）表示的直观的天气的自相关性：如果当前是晴天，则天气倾向于保持晴朗；如果是多云则倾向于保持多云，等等。这种模式经常在马尔可夫模型中出现，体现在状态转移矩阵中就是对角线元素远大于其他元素。在本例中，初始状态$s_0$是转换到这三个天气状态上的均匀概率分布。

### 1.1 马尔可夫模型的两个问题

结合马尔可夫假设和状态转换参数$A$，我们可以回答在马尔科夫链中关于状态序列的两个基本问题：状态$\vec z$对应的一个特定序列的概率是什么？对于观测到的序列$\vec z$，如何估计模型中的参数$A$的才能最大化其似然性？

\left(\right)
\left\{\right\}